In [1]:
import ase
from ase.build import fcc111, fcc100, fcc110
from moleidoscope.visualize import show
from moleidoscope.linker import Linker
from xyz2cif import write_cif

In [ ]:
def ase_to_linker(ase_atoms):
    """ Convert ASE atoms object to Moleidoscope linker object """
    mol = Linker()
    mol.atom_names = list(ase_atoms.get_chemical_symbols())
    mol.atom_coors = list(ase_atoms.get_positions())
    return mol

# Parameters

In [ ]:
slab_packing = (10, 15, 5)    # Packing of metal slab
vacuum = 0.0                  # Vacuum on top of slab (ASE)
z_box = 50                    # Length of simulation box in z-direction (A)
distance_from_surface = 5     # Distance btw molecule and surface (A)

## Cu (110) Slab

In [ ]:
cu_slab = fcc110('Cu', a=3.908, size=slab_packing, vacuum=vacuum)
cu110 = ase_to_linker(cu_slab)

In [ ]:
# show(cu110)

### Aligning Decacylene (*xy-plane*)
Original xyz file already has one head (atoms 7 & 8) aligned to *y-axis*.

Here I am using the vector between the two opposing atoms in the middle benzene ring that is perpendicular to the head on *y-axis*.

By aligning this vector to *x-axis* the molecule is now aligned to *xy-plane*.

In [ ]:
decacylene = Linker(read='DC-single.xyz')            # Read molecule
decacylene.vector = decacylene.get_vector(0, 1)      # Get vector for middle benzene ring atoms
dc_align = decacylene.align([1, 0, 0])               # Align molecule on x-axis
dc_align.center([0, 0, 0])                           # Center molecule to origin

In [ ]:
# show(dc_align, decacylene, distance=[20, 0, 0])

In [ ]:
dc_align.name='DC-align'           # Rename molecule
dc_align.save(file_format='xyz')   # Save as xyz file

### Placing aligned decacylene on Cu (110) surface
I will place the aligned decacylene 5 Å above the Cu (110) surface.

In [ ]:
slab_center = cu110.get_center()
dc_align.center(slab_center)
dc_align.translate([0, 0, distance_from_surface + cu_slab.cell[2][2] / 2])
cu110_dc = cu110.join(dc_align)

In [ ]:
show(cu110_dc)

In [ ]:
cu110_dc.name='DC_Cu110'           # Rename molecule
cu110_dc.save(file_format='xyz')   # Save as xyz file

## Create cif file

In [ ]:
cif_file = 'DC_Cu110.cif'
sim_box = [cu_slab.cell[0][0], cu_slab.cell[1][1], z_box, 90, 90, 90]
write_cif(cif_file, cu110_dc.atom_names, cu110_dc.atom_coors, header=cif_file, cell=sim_box, fractional=True)

## Create Lammps simulation files